<a href="https://colab.research.google.com/github/10anshuman/Ben-LLM/blob/main/4_bit_bengali_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets loralib sentencepiece
!pip install -q gradio
!pip install -q ttsmms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [ ]:
# For TTS

!curl https://dl.fbaipublicfiles.com/mms/tts/ben.tar.gz --output ben.tar.gz #update lang
!mkdir -p data && tar -xzf ben.tar.gz -C data/ #update langcode

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0   142M      0 --:--:-- --:--:-- --:--:--  142M


In [ ]:
from ttsmms import TTS

tts=TTS("data/ben") #update lang code

wav=tts.synthesis("আমি অলিভ, ওড়িয়া জেনারেল এআই গবেষকদের দ্বারা প্রশিক্ষিত একটি ভাষা মডেল")


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.11/dist-packages/ttsmms/utils.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=

In [ ]:
from IPython.display import Audio
Audio(wav["x"], rate=wav["sampling_rate"])

In [ ]:
import tempfile
from IPython.display import Audio


def tts_system(text: str):

    print(text)
    wavs = tts.synthesis(text)
    # return output
    af = Audio(wavs["x"], rate=wavs["sampling_rate"])
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as fp:
        fp.write(af.data)
        return fp.name

In [ ]:
# import torch
from peft import PeftModel
# from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig


import torch
# from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, BitsAndBytesConfig

In [ ]:
model_id = "OdiaGenAI/odiagenAI-bengali-base-model-v1"
device = "cuda:0"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")   # 4 bit model

def generate_prompt(instruction, input=None):
    if input:
        return f"""### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"""
    else:
        return f"""### Instruction:\n{instruction}\n\n### Response:\n"""


model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    print(inputs)
    input_ids = inputs["input_ids"].to(device)
    print(input_ids)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
        # print(generation_output)
    s = generation_output.sequences[0]
    print(s)
    output = tokenizer.decode(s, skip_special_tokens=True)
    print(output)

    text = output.split("### Response:")[1].strip()
    audio_wav = tts_system(text)
    return text, audio_wav
    # return output.split("### Response:")[1].strip()

input_text = gr.components.Textbox(
            lines=2, label="Instruction", placeholder="Tell me about alpacas."
        )
# tts_button = gr.Button("Change to voice", elem_id="send-btn", visible=True)

output_audio = gr.Audio(label="Output", type="filepath")

g = gr.Interface(
    fn=evaluate,
    inputs=[
        input_text,
        gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(minimum=0, maximum=1, value=0.1, label="Temperature"),
        gr.components.Slider(minimum=0, maximum=1, value=0.75, label="Top p"),
        gr.components.Slider(minimum=0, maximum=100, step=1, value=40, label="Top k"),
        gr.components.Slider(minimum=1, maximum=4, step=1, value=4, label="Beams"),
        gr.components.Slider(
            minimum=1, maximum=512, step=1, value=128, label="Max tokens"
        ),
    ],
    outputs=[
        gr.Textbox(
            lines=5,
            label="Output",
        ),
        output_audio
    ],
    title="🌊🐢 OdiaGenAI-4-bit-Bengali-LLM",
    description="OdiaGenAI-4-bit-Bengali-LLM is a 7B-parameter LLaMA model finetuned to follow Odia instructions. It is trained on the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) Odia translated dataset and makes use of the Huggingface LLaMA implementation. For more information, please visit [the project's website](https://www.odiagenai.org/).",
)



# g.queue(concurrency_count=2)
g.launch(server_name="0.0.0.0", share= True, debug=True, server_port= 8000)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:604: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://51d75710aa3f9ac5d8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Instruction:
আমার নাম আনশুমান মিশরা, আমি মাথা ব্যাথা করে ভুগছি তাই আপনি আমাকে কিছু ওষুধের পরামর্শ দিতে পারেন, দয়া করে

### Response:

{'input_ids': tensor([[    0,   835,  2799,  4080, 29901,    13,   227,   169,   137, 31006,
         30445, 30653, 29871, 30792, 30445, 31006, 29871,   227,   169,   137,
         30792, 31686, 31154, 31006, 30445, 30792, 29871, 31006, 30786, 31686,
         30653, 30445, 29892, 29871,   227,   169,   137, 31006, 30786, 29871,
         31006, 30445,   227,   169,   168, 30445, 29871, 30962, 30655, 31218,
         30445,   227,   169,   168, 30445, 29871, 30995, 30653, 30932, 29871,
           227,   169,   176, 31154, 31665,   227,   169,   158, 30786, 29871,
         31150, 30445,   227,   169,   138, 29871,   227,   169,   137, 31386,
         30792, 30786, 29871,   227,   169,   137, 31006, 30445, 30995, 30932,
         29871, 30995, 30786,   227,   169,   158, 31154, 29871,   227,   169,
           150,   227,   169,   186, 31154,   227,   169,

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:626: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configurati

tensor([    0,   835,  2799,  4080, 29901,    13,   227,   169,   137, 31006,
        30445, 30653, 29871, 30792, 30445, 31006, 29871,   227,   169,   137,
        30792, 31686, 31154, 31006, 30445, 30792, 29871, 31006, 30786, 31686,
        30653, 30445, 29892, 29871,   227,   169,   137, 31006, 30786, 29871,
        31006, 30445,   227,   169,   168, 30445, 29871, 30962, 30655, 31218,
        30445,   227,   169,   168, 30445, 29871, 30995, 30653, 30932, 29871,
          227,   169,   176, 31154, 31665,   227,   169,   158, 30786, 29871,
        31150, 30445,   227,   169,   138, 29871,   227,   169,   137, 31386,
        30792, 30786, 29871,   227,   169,   137, 31006, 30445, 30995, 30932,
        29871, 30995, 30786,   227,   169,   158, 31154, 29871,   227,   169,
          150,   227,   169,   186, 31154,   227,   169,   170, 30932, 30653,
        29871, 31386, 30653, 30445, 31006, 30653, 30655, 31686, 29871, 31204,
        30786, 31150, 30932, 29871, 31386, 30445, 30653, 30932, 